 # Designing an API Rate Limiter

An API reate limiter will throttle users based on the number of requests they are sending.

## Why Rate Limiting?
Rate limiting helps to protect services against abusive behaviors targeting the application such as Denial-of-service (DOS) attacks, brute-force password attempts, brute-force credit card transactions, etc.

We also want to prevent revenue loss, to reduce infrastructure costs, stop spamming and stop online harassment. 

Here's some scenarios that show how benefitial it is to Rate limit our API/Service:

- **Misbehaving clients:** Sometimes, clients can overwhelm servers by sending large number of requests, either intentionally or unintentionally. 

- **Security:** Limiting the number of times a user is allowed to try authenticating with a wrong password.

- **Preventing abusive and bad design practices:** Without API limits, developers of client apps might request the same info over and over again.

- **Revenue:**  Certain services might want to limit operations based on the tier of their customer's service and thus create a revenue model off the rate limiting. To go beyond the set limit, the user has to buy higher limits.

- Prevent spikiness of traffic so that the service stays reliably up for all.


## 1. Requirements and System Goals

#### Functional requirements
1. Limit the number of requests an entity can send to an API within a time window
2. The user should get an error whenever they cross the defined threshold within a single server or across a set of servers.

#### Non-Functional requirements
1. The system should be highly available, always protecting our API service from external attacks.
2. The rate limiter should NOT introduce substantial latencies affecting the user experience.

## 2. Throttling Types

* ***Hard Throttling*** – Number of API requests cannot exceed the throttle limit

* ***Soft Throttling*** – Set the API request limit to exceed by some percentage. E.g, if the rate-limit = 100 messages/minute, and 10% exceed-limit, our rate limiter will allow up to 110 messages per minute

* ***Dynamic Throttling*** – The number of requests can exceed the limit if the system has some free resources available. 

## 3. Algorithms used for Rate Limiting

#### Fixed Window Algorithm
Here, the time window is considered from the start of the time-unit to the end of the time-unit. For instance, a period will be considered 0-60 sec for a minute regardless of the time frame at which the API request has been made.

The diagram below shows that we will only throttle 'm5' message, if our rate limit is 2 messages per second.

![](images/fixed_rolling_window.svg)

#### Rolling Window Algorithm
Here, the time window is considered from the fraction of time at which the request is made plus the time window length.

For example, if our rate limit = 2 msg per sec, the two messages sent at the 300th millisecond (m1) and 400th millisecond (m2), we'll count them as two messages starting from the 300th of that second to the 300th of the next second (making up one second).

In the above diagram, we'll therefore throttle m3, m4.




## 4. High Level Design

Once a new request arrives, the Web server first asks the Rate Limiter to decide if it will be served or throttled. If the request is not throttled, the it's passed to the API servers. 

![](images/rate_limiter_high_level_design.png)


## 5. Basic System Design and Algorithm

Assume our rate limiter allows 3 requests/sec per user. 

For each unique user, 
- Keep a count representing how many requests the user has made
- and a timestamp when we started counting

We can keep this in a hashtable, where:
```python
#  Key (userID): Value {count, start_time}
hashtable = {
    'userID0': {
        'count': 3, 'start_time': 1574866492
    },
    'userId1': {
        'count': 1, 'start_time': 1574873716
    },
    ...
}
```
When a new request comes in, the rate limiter will perform the following steps:

1. If the `userID` is not present in hash-table, 
    - insert it, 
    - set `count` to 1 and set `start_time` to current epoch time
2. Otherwise, find the existing record of the userID, and 
    - if `current_time - start_time >= 1 min`, set the `start_time` to be the current time, 
    - set `count` to 1 and allow the request
3. If `current_time - start_time <= 1 min` and
    - If `count < 3`, increment the count and allow the request.
    - If `count >= 3`, reject the request.
    
#### Problems with this Fixed Window Algorithm
1. We are resetting the `start_time` at the end of every minute, which means we can potentially allow twice the number of requests per minute.

Imagine if a user sends 3 requests at the last second of a minute, they can immediately send 3 more requests at the very first second of the next minute, resulting in 6 requests in a span of two seconds. The solution for this would be a sliding window algorithm.

![](images/fixed_window_problem.svg)

2. Atomicity: The read and then write process can create a race condition. Imagine, a given user's current count = 2. If two seperate processes served each of these requests and concurrently read the count before either updated it, each process would erroneously think that the user had one more request to go hit the rate limit. 

![](images/fixed_window_atomicity.svg)

## 6. Slide Window Algorithm
